In [1]:
from model.MF import *
from preprocess.AmazonBook import *
from evaluation.MF_evaluation import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/amazon-book'
dataset = AmazonBook(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'learning_rate': 1e-5,  # over-fitting
    'epochs': 120,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
    "weight_decay": 1e-7,
    'global_bias':(data.edge_index.size(1) + data.edge_label_index.size(1) + 2) / (num_books * num_users)
}
model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

# Split the dataset

In [2]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

# Retain MF model

In [3]:
config['epochs'] = 120
retrain_model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)
retrain_model, recall, ndcg = MF_based_eva(retrain_model, config, retain_data, device)

Epoch 1/120, Train Loss: 0.9967, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 2/120, Train Loss: 0.9924, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 3/120, Train Loss: 0.9880, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 4/120, Train Loss: 0.9837, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 5/120, Train Loss: 0.9794, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 6/120, Train Loss: 0.9750, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 7/120, Train Loss: 0.9707, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch 8/120, Train Loss: 0.9664, HR@20: 0.0001, Recall@20: 0.0003, NDCG@20: 0.0002
Epoch 9/120, Train Loss: 0.9621, HR@20: 0.0002, Recall@20: 0.0003, NDCG@20: 0.0002
Epoch 10/120, Train Loss: 0.9578, HR@20: 0.0002, Recall@20: 0.0003, NDCG@20: 0.0003
Epoch 11/120, Train Loss: 0.9536, HR@20: 0.0002, Recall@20: 0.0004, NDCG@20: 0.0004
Epoch 12/120, Train Loss: 0.9493, HR@20: 0.0004, Recall@20: 0.0006, NDCG@20: 0.0006
E

In [4]:
MF_forget_data_eva(retrain_model, None, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0044, Recall@20: 0.0241, NDCG@20: 0.0131


# Prompt Unlearning
## Case 1: Without Contrastive Loss and Regularization

In [5]:
# Define the model
teacher = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)
student = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

# Load the model
teacher.load_state_dict(torch.load(f"MF_Amazon_Book_{config['epochs']}_Epochs_Top_{config['k']}.pt"))
student.load_state_dict(torch.load(f"MF_Amazon_Book_{config['epochs']}_Epochs_Top_{config['k']}.pt"))

<All keys matched successfully>

In [6]:
# Setting the basic hyperparameters
config['beta'] = 0.7
config['alpha'] = 0.3
config['epochs'] = 50
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-3 # regularization loss
config['tuning_type'] = 'gpf'
config['learning_rate'] = 1e-5
config['weight_decay'] = 0
config['regularization'] = False
config['Contrastive_loss'] = False
student, prompt= prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

c:\Users\MSI\.conda\envs\master\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'x', 'edge_label_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Epoch 1/50, Train Loss: 0.6919, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 2/50, Train Loss: 0.6918, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 3/50, Train Loss: 0.6917, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 4/50, Train Loss: 0.6917, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 5/50, Train Loss: 0.6916, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 6/50, Train Loss: 0.6915, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 7/50, Train Loss: 0.6915, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 8/50, Train Loss: 0.6914, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 9/50, Train Loss: 0.6913, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 10/50, Train Loss: 0.6912, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 11/50, Train Loss: 0.6911, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 12/50, Train Loss: 0.6911, HR@20: 0.0046, Recall@20: 0.0097, NDCG@20: 0.0085
Epoch 13/50, 

In [7]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0025, Recall@20: 0.0132, NDCG@20: 0.0069


## Case 2: Without Contrastive Loss but with Regularization

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = False
config['regularization'] = True

student, prompt = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

## Case 3: With Contrastive Loss but without Regularization

In [10]:
# Setting the basic hyperparameters
config['Contrastive_loss'] = True
config['regularization'] = False
config['gamma'] = 1e-5

student, prompt= prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

Epoch 1/50, Train Loss: 3.9762, HR@20: 0.0045, Recall@20: 0.0094, NDCG@20: 0.0083
Epoch 2/50, Train Loss: 3.9635, HR@20: 0.0045, Recall@20: 0.0094, NDCG@20: 0.0083
Epoch 3/50, Train Loss: 3.9508, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 4/50, Train Loss: 3.9382, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 5/50, Train Loss: 3.9256, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 6/50, Train Loss: 3.9130, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 7/50, Train Loss: 3.9005, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 8/50, Train Loss: 3.8881, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 9/50, Train Loss: 3.8757, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 10/50, Train Loss: 3.8634, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 11/50, Train Loss: 3.8511, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 12/50, Train Loss: 3.8389, HR@20: 0.0045, Recall@20: 0.0095, NDCG@20: 0.0083
Epoch 13/50, 

In [9]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

HR@20: 0.0030, Recall@20: 0.0151, NDCG@20: 0.0086


## Case 4: With Contrastive Loss and Regularization

In [ ]:
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)

# Additional Materials(Multiple Prompts)

In [ ]:
config['tuning_type'] = 'gpf-plus'
config["number_p"] = 10 # The number of prompts
# Setting the basic hyperparameters
config['contrastive_loss'] = True
config['regularization'] = True

student, prompt, epoch_tracks, test_topks = prompt_MF_unlearning_eva(teacher, student, config, retain_data, forget_data, device)

Epoch 1/50, Train Loss: 0.6037, HR@20: 0.0068, Recall@20: 0.0147, NDCG@20: 0.0123
Epoch 2/50, Train Loss: 0.6037, HR@20: 0.0068, Recall@20: 0.0147, NDCG@20: 0.0123
Epoch 3/50, Train Loss: 0.6037, HR@20: 0.0068, Recall@20: 0.0147, NDCG@20: 0.0123
Epoch 4/50, Train Loss: 0.6037, HR@20: 0.0067, Recall@20: 0.0147, NDCG@20: 0.0123
Epoch 5/50, Train Loss: 0.6037, HR@20: 0.0067, Recall@20: 0.0147, NDCG@20: 0.0123
Epoch 6/50, Train Loss: 0.6036, HR@20: 0.0067, Recall@20: 0.0147, NDCG@20: 0.0123
Epoch 7/50, Train Loss: 0.6036, HR@20: 0.0067, Recall@20: 0.0146, NDCG@20: 0.0123
Epoch 8/50, Train Loss: 0.6036, HR@20: 0.0067, Recall@20: 0.0146, NDCG@20: 0.0123
Epoch 9/50, Train Loss: 0.6036, HR@20: 0.0067, Recall@20: 0.0146, NDCG@20: 0.0123
Epoch 10/50, Train Loss: 0.6036, HR@20: 0.0067, Recall@20: 0.0146, NDCG@20: 0.0123
Epoch 11/50, Train Loss: 0.6036, HR@20: 0.0067, Recall@20: 0.0146, NDCG@20: 0.0123
Epoch 12/50, Train Loss: 0.6035, HR@20: 0.0067, Recall@20: 0.0146, NDCG@20: 0.0123
Epoch 13/50, 

In [ ]:
MF_forget_data_eva(student, prompt, forget_data, num_users, config['k'], config['batch_size'], device)